<a href="https://www.kaggle.com/code/hossamahmedsalah/icr-eda?scriptVersionId=137492330" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as ply
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.subplots as sp




/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Loading the data

In [2]:
train = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/greeks.csv')
test = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')
greek = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/greeks.csv')

> Train dataset
> 
> - `Id` Unique identifier for each observation.
> - `AB``GL` Fifty-six anonymized health characteristics. All are numeric except for `EJ`, which is categorical.
> - `Class` A binary target: `1` indicates the subject has been diagnosed with one of the three conditions, `0` indicates they have not.

> Test dataset
> 
> 
> the actual test set **is hidden**. we only it’s format with no real data example test data will be replaced with the full test set. **There are about 400 rows in the full test set.**
> 

> greeks
> 
> 
> Supplemental metadata, only available for the training set.
> 
> - `Alpha` Identifies the type of age-related condition, if present.
>     - `A` No age-related condition. Corresponds to class `0`.
>     - `B`, `D`, `G` The three age-related conditions. Correspond to class `1`.
> - `Beta`, `Gamma`, `Delta` Three experimental characteristics.
> - `Epsilon` The date the data for this subject was collected. Note that all of the data in the test set was collected after the training set was collected.

In [3]:
display(train, test, greek)

,Id,Alpha,Beta,Gamma,Delta,Epsilon
0,000ff2bfdfe9,B,C,G,D,3/19/2019
1,007255e47698,A,C,M,B,Unknown
2,013f2bd269f5,A,C,M,B,Unknown
3,043ac50845d5,A,C,M,B,Unknown
4,044fb8a146ec,D,B,F,B,3/25/2020
...,...,...,...,...,...,...
612,fd3dafe738fd,A,B,M,B,9/13/2020
613,fd895603f071,A,B,M,B,9/8/2020
614,fd8ef6377f76,A,C,M,B,7/24/2019
615,fe1942975e40,A,C,M,B,1/31/2019


,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL
0,00eed32682bb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,010ebe33f668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,02fa521e1838,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,040e15f562a2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,046e85c7cc7f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,Id,Alpha,Beta,Gamma,Delta,Epsilon
0,000ff2bfdfe9,B,C,G,D,3/19/2019
1,007255e47698,A,C,M,B,Unknown
2,013f2bd269f5,A,C,M,B,Unknown
3,043ac50845d5,A,C,M,B,Unknown
4,044fb8a146ec,D,B,F,B,3/25/2020
...,...,...,...,...,...,...
612,fd3dafe738fd,A,B,M,B,9/13/2020
613,fd895603f071,A,B,M,B,9/8/2020
614,fd8ef6377f76,A,C,M,B,7/24/2019
615,fe1942975e40,A,C,M,B,1/31/2019


> Let’s start by understanding the **greek** set to notice the patterns  between age related conditions **A is same as class 0 , means that he is ok  but if Alpha is  B, D or G he has an age related condition**
>

In [4]:
greek.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 617 entries, 0 to 616
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Id       617 non-null    object
 1   Alpha    617 non-null    object
 2   Beta     617 non-null    object
 3   Gamma    617 non-null    object
 4   Delta    617 non-null    object
 5   Epsilon  617 non-null    object
dtypes: object(6)
memory usage: 29.0+ KB


# greek analysis 

In [5]:
greek.describe()

,Id,Alpha,Beta,Gamma,Delta,Epsilon
count,617,617,617,617,617,617
unique,617,4,3,8,4,198
top,000ff2bfdfe9,A,C,M,B,Unknown
freq,1,509,407,445,456,144


> we understand the `Aplha` four classes {A✅82.5%, B❌ , D❌ , G❌}
> 
> 
> `Beta` has 3 classes with `C` is the top with 407 ⇒65.9%
> 
> `Gama` has 8 classes with `M` is the top 445 ⇒72.1%
> 
> `Delta` has 4 classes with `B` is the top 456 ⇒ 73.9%
> 
> `Epsilon` 198 different days and 144 unknown dates of collecting these data, *the test data Epsilon would be different as it gathered after the training set*  
>

> most of the classes are unbalanced completely to a be for only one class is greater than 60% to 70% of the distribution
> 
> - we have 82.5% oof  people that has no age related condition so maybe that is why all the other classes have a common class maybe that is mean the person is less likely to have age related issues

In [6]:
greek.columns

Index(['Id', 'Alpha', 'Beta', 'Gamma', 'Delta', 'Epsilon'], dtype='object')

In [7]:
cols = ['Alpha', 'Beta', 'Gamma', 'Delta', 'Epsilon']
for col in cols:
    print("+"*50, f"{col}", "+"*50)
    print(greek[col].value_counts())
    if col != 'Epsilon':
        labels = greek[col].value_counts().index.tolist()
        values = greek[col].value_counts().values.tolist()
        trace = go.Pie(labels=labels, values=values)
        layout = go.Layout(title=f"Class distribution for {col}")
        fig = go.Figure(data=[trace], layout=layout)
        fig.show()
  
   

++++++++++++++++++++++++++++++++++++++++++++++++++ Alpha ++++++++++++++++++++++++++++++++++++++++++++++++++
A    509
B     61
G     29
D     18
Name: Alpha, dtype: int64


++++++++++++++++++++++++++++++++++++++++++++++++++ Beta ++++++++++++++++++++++++++++++++++++++++++++++++++
C    407
B    202
A      8
Name: Beta, dtype: int64


++++++++++++++++++++++++++++++++++++++++++++++++++ Gamma ++++++++++++++++++++++++++++++++++++++++++++++++++
M    445
N     64
H     53
B     18
A     11
F     10
G      8
E      8
Name: Gamma, dtype: int64


++++++++++++++++++++++++++++++++++++++++++++++++++ Delta ++++++++++++++++++++++++++++++++++++++++++++++++++
B    456
A     75
C     64
D     22
Name: Delta, dtype: int64


++++++++++++++++++++++++++++++++++++++++++++++++++ Epsilon ++++++++++++++++++++++++++++++++++++++++++++++++++
Unknown       144
5/29/2019      24
6/26/2019      19
9/15/2020      17
8/29/2019      13
             ... 
2/21/2019       1
8/13/2019       1
5/18/2020       1
12/20/2019      1
7/24/2019       1
Name: Epsilon, Length: 198, dtype: int64


In [8]:

# Define the columns
columns = ["Alpha", "Beta", "Gamma", "Delta"]

# Create subplots with treemap type for each column
fig = make_subplots(rows=1, cols=4, specs=[[{"type": "treemap"}, {"type": "treemap"}, {"type": "treemap"}, {"type": "treemap"}]],horizontal_spacing=0.0008, vertical_spacing=0.0008)

# Define trace variables for each column using loops
traces = []
for i, column in enumerate(columns):
    labels = greek[column].value_counts().index.tolist()
    parents = [columns[i] for j in range(len(labels))]
    values = greek[column].value_counts().values.tolist()
    trace = go.Treemap(labels=labels, parents=parents, values=values)
    traces.append(trace)
    fig.add_trace(trace, row=1, col=i+1)

# Update subplot layout to minimize spacing
fig.update_layout(height=500, margin={"t":0, "b":0, "l":0, "r":0})

fig.show()

# Train data analysis 

In [9]:
train.shape

(617, 6)

In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 617 entries, 0 to 616
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Id       617 non-null    object
 1   Alpha    617 non-null    object
 2   Beta     617 non-null    object
 3   Gamma    617 non-null    object
 4   Delta    617 non-null    object
 5   Epsilon  617 non-null    object
dtypes: object(6)
memory usage: 29.0+ KB


In [11]:
train.describe()

,Id,Alpha,Beta,Gamma,Delta,Epsilon
count,617,617,617,617,617,617
unique,617,4,3,8,4,198
top,000ff2bfdfe9,A,C,M,B,Unknown
freq,1,509,407,445,456,144


In [12]:
train.describe(include='O')

,Id,Alpha,Beta,Gamma,Delta,Epsilon
count,617,617,617,617,617,617
unique,617,4,3,8,4,198
top,000ff2bfdfe9,A,C,M,B,Unknown
freq,1,509,407,445,456,144


# Using integerized set 
> because the dataset is [annomized](!https://hossam-ahmed.notion.site/Anonymizing-Data-Sets-8c413548af914a0282598d1baca361cb?pvs=4) its more computational to work with float64 (in visualising also) so I would use the work of [raddar](!https://www.kaggle.com/raddar) in this
>[notebook](!https://www.kaggle.com/code/raddar/convert-icr-data-to-integers) he find denominators for each feature to rescalle it better to integers-like values he explains how he did it in this [comment](!https://www.kaggle.com/code/raddar/convert-icr-data-to-integers/comments#2324153)


In [13]:
# reading the intergrized set 
int_train = pd.read_csv('/kaggle/input/icr-integer-data/train_integerized.csv')
int_train


,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,000ff2bfdfe9,49.0,1284724.5,9783.0,7231.0,1552.0,79.0,42.0,1557.0,790.5,...,7.298162,2997.5,14.0,1219.0,16439.0,458118.5,5949.0,27152.0,0.120343,1
1,007255e47698,34.0,404448.0,9783.0,11937.0,1552.0,410.0,42.0,2145.0,175.0,...,0.173229,857.0,84.0,999.0,16439.0,6397248.0,7830.0,12493.0,21.978000,0
2,013f2bd269f5,110.0,1088887.0,9783.0,10449.0,1552.0,760.0,42.0,2035.0,175.0,...,7.709560,1682.0,177.0,3986.0,20061.0,3126876.5,7531.0,13683.0,0.196941,0
3,043ac50845d5,59.0,1578368.5,13802.0,24899.0,1552.0,416.0,42.0,1754.0,175.0,...,6.122162,857.0,42.0,1992.0,18659.0,478798.0,10736.0,35184.0,0.155829,0
4,044fb8a146ec,89.0,1542582.0,9783.0,4554.0,1552.0,445.0,90.0,539.0,14535.0,...,8.153058,83623.0,18.0,1764.0,33079.0,1948873.0,12196.0,14099.0,0.096614,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612,fd3dafe738fd,35.0,1293413.0,14211.0,3072.0,2483.0,395.0,127.0,1356.0,399.0,...,0.173229,2174.0,10.0,964.0,49162.0,1850922.0,6622.0,26902.0,21.978000,0
613,fd895603f071,102.0,2257039.0,9783.0,15031.0,3046.0,675.0,42.5,2003.0,537.5,...,10.223150,2142.0,63.0,3859.0,112518.5,705374.5,7968.0,48526.0,0.145340,0
614,fd8ef6377f76,100.0,1016160.0,14943.0,17874.0,1908.0,911.0,42.0,2445.0,175.0,...,0.173229,857.0,10.0,2146.0,29182.0,1480259.0,7032.0,46485.0,21.978000,0
615,fe1942975e40,85.0,522122.0,9783.0,7648.0,1552.0,901.0,42.0,1194.0,175.0,...,9.256996,1358.0,99.0,2644.0,16439.0,449324.0,6750.0,14446.0,0.184622,0


In [14]:
labels = int_train['Class'].value_counts().index.tolist()
values = int_train['Class'].value_counts().values.tolist()
trace = go.Pie(labels = labels, values = values)
layout = go.Layout(title="Distribution of Target classes")
fig = go.Figure(data=[trace], layout=layout)
fig.show()


In [15]:

target_column = 'Class'

# Create a list of column names to plot
columns_to_plot = [column for column in int_train.columns if column not in [target_column, 'Id']]

# Define the number of rows and columns for the subplot grid
num_rows = 10
num_cols = 6

# Create a subplot grid with Plotly
fig = sp.make_subplots(rows=num_rows, cols=num_cols, subplot_titles=columns_to_plot)

# Loop over each column and add a histogram trace to the subplot grid
for i, column in enumerate(columns_to_plot):
    row_num = (i // num_cols) + 1
    col_num = (i % num_cols) + 1
    trace = go.Histogram(x=int_train[column], nbinsx=60, name=column, marker=dict(color=int_train[target_column].replace({1: 'salmon', 0: 'skyblue'})))
    fig.add_trace(trace, row=row_num, col=col_num)

# Update the layout of the subplot grid
fig.update_layout(height=1500, width=1500, showlegend=False)

fig.show()
# 🔵 for class 0
# 🔴 for class 1

>so we can note that some features is more affecting one class more 
> and in others the red class=1 is more like spikes 

    

In [16]:
corr = int_train.drop('Id', axis=1).corr()


/tmp/ipykernel_21/4242857130.py:1: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



In [17]:
corr.iloc[:10, :10]

,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD
AB,1.000000,0.350231,0.249246,0.530687,0.157712,0.471912,0.011004,0.158569,0.331736,0.278920
AF,0.350231,1.000000,0.044140,0.183961,0.044917,0.248439,0.039417,0.219172,0.058831,0.045430
AH,0.249246,0.044140,1.000000,0.128268,0.749794,0.039324,0.055385,0.095627,0.366910,0.209508
AM,0.530687,0.183961,0.128268,1.000000,0.175212,0.225382,0.081666,0.131885,0.329658,0.277908
AR,0.157712,0.044917,0.749794,0.175212,1.000000,0.011715,0.099947,0.099282,0.073730,-0.002509
AX,0.471912,0.248439,0.039324,0.225382,0.011715,1.000000,0.016315,0.269728,0.079799,0.124581
AY,0.011004,0.039417,0.055385,0.081666,0.099947,0.016315,1.000000,0.036790,0.132495,0.506917
AZ,0.158569,0.219172,0.095627,0.131885,0.099282,0.269728,0.036790,1.000000,0.036424,0.135976
BC,0.331736,0.058831,0.366910,0.329658,0.073730,0.079799,0.132495,0.036424,1.000000,0.753051
BD,0.278920,0.045430,0.209508,0.277908,-0.002509,0.124581,0.506917,0.135976,0.753051,1.000000


In [18]:
labels = [col for col in int_train.columns if col not in ['Id', 'Class']]

trace = go.Heatmap(z=corr, x=labels, y=labels, colorscale='RdBu', zmin=-1, zmax=1)

# Create the figure layout
layout = go.Layout(title='Correlation Matrix Heatmap', xaxis=dict(title='Variables', tickangle=45), yaxis=dict(title='Variables'))

# Create the figure object
fig = go.Figure(data=[trace], layout=layout)

fig.show()